In [1]:
import atlite
import cartopy.io.shapereader as shpreader
import geopandas as gpd
from shapely.geometry import box
import pandas as pd
import logging
from argparse import ArgumentParser
from datetime import datetime

In [2]:
print(f'0 : {datetime.now()}')

year = '2011'

logging.basicConfig(level=logging.INFO)

world = gpd.read_file("./data/ne_110m_admin_0_countries.shp")

# Drop uninhabited regions and Antarctica
# world = world[(world.POP_EST > 0) & (world.NAME != "Antarctica")]

# region = world
region_name = "western-europe"

# Loop over the years
logging.info(f"Processing {year}")

INFO:root:Processing 2011


0 : 2024-10-06 18:04:27.170372


In [3]:
print(f'1 : {datetime.now()}')

# Define the cutout; this will not yet trigger any major operations
cutout = atlite.Cutout(
    path=f"{region_name}-{year}-01.nc",
    module="era5", 
    # bounds=region.unary_union.bounds, 
    x=slice(-13.6913, 1.7712),
    y=slice(49.9096, 60.8479),
    time=f"{year}-01",
)

INFO:atlite.cutout:Building new cutout western-europe-2011-01.nc


1 : 2024-10-06 18:04:29.632189


In [4]:
print(cutout)

<Cutout "western-europe-2011-01">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2011-01-01 ⟷ 2011-01-31, dt = h
 module = era5
 prepared_features = []


In [8]:
# This is where all the work happens (this can take some time).
cutout.prepare(
    compression={"zlib": True, "complevel": 9},
    features = ['wind',],
    monthly_requests=True,
    concurrent_requests=True,
)

print(f'2 : {datetime.now()}')

INFO:atlite.data:Storing temporary files in /tmp/tmp5xwvvt71


2 : 2024-10-06 18:07:08.489006


In [9]:
# Extract the wind power generation capacity factors
wind_power_generation = cutout.wind(
    "Vestas_V112_3MW", 
    capacity_factor_timeseries=True,
    )

print(f'3 : {datetime.now()}')
print(wind_power_generation)

/home/codespace/.python/current/lib/python3.12/site-packages/atlite/resource.py:72: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.15.
  warnings.warn(msg, FutureWarning)
INFO:atlite.convert:Convert and aggregate 'wind'.


3 : 2024-10-06 18:07:11.570450
<xarray.DataArray 'capacity factor' (time: 744, y: 44, x: 62)> Size: 16MB
array([[[2.07162156e-01, 2.03869087e-01, 1.78809979e-01, ...,
         8.57697593e-04, 4.87033059e-04, 1.82842118e-05],
        [1.69812990e-01, 1.55065223e-01, 1.41594819e-01, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.44329550e-01, 1.26802419e-01, 1.16621980e-01, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [7.36809328e-01, 6.99846847e-01, 6.71577921e-01, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [6.66657130e-01, 6.35407851e-01, 6.15444939e-01, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [6.25561873e-01, 5.98668803e-01, 5.86504290e-01, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

       [[1.83471508e-01, 1.68532262e-01, 1.45077659e-01, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.49398750e-01, 1.33870022e-01,